In [49]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

loader = UnstructuredFileLoader("document.txt")
documents = loader.load()

splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=0, separator="\n",)
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(docs, embeddings)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(return_messages=True)

llm = ChatOpenAI(temperature=0.1)

prompt = ChatPromptTemplate.from_messages([    
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}"),
    MessagesPlaceholder(variable_name="history")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory, 
    }
    | prompt
    | llm
)

In [50]:
questions = [
    "Is Aaronson guilty?",
    "What message did he write on the table?",
    "Who is Julia?"
]

for question in questions:
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

    

content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'
content='He wrote "2+2=5" on the table.'


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-RyIXvx5ywrEb86kgNEgndnrX on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-RyIXvx5ywrEb86kgNEgndnrX on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.